In [11]:
import os

# Get the current working directory
cwd = os.getcwd()
print("Current working directory:", cwd)

# List all files and folders in the current directory
files = os.listdir(cwd)
print("Files in directory:", files)


Current working directory: C:\Users\USER\OneDrive - University of Plymouth\COMP5012\Course Work\Report - Optimisation\Boxes on shelves
Files in directory: ['.ipynb_checkpoints', 'baytp1.txt', 'baytp2.txt', 'COMP5012 - Boxes on shelves.ipynb', 'Final Submission Attempt', 'Optimizing Shelf Placement - A MOGA Approach.docx', 'products.txt', 'shelves.txt']


In [19]:
!pip install deap
import random
import numpy as np
import deap
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt


# Load baytp1.txt and baytp2.txt
def load_bays(filename):
    bays = []
    try:
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                values = line.split()
                if len(values) != 4:
                    raise ValueError(f"Invalid line in {filename}: '{line}' (expected 4 values)")
                try:
                    width, height, depth, avail_height = map(int, values)
                    bays.append({"width": width, "height": height, "depth": depth, "avail_height": avail_height})
                except ValueError as e:
                    raise ValueError(f"Non-integer values in {filename}: '{line}'")
        if not bays:
            raise ValueError(f"{filename} is empty")
        print(f"Loaded {len(bays)} bays from {filename}")
        return bays
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find {filename} in {os.getcwd()}")

# Load files
try:
    bays_tp1 = load_bays("baytp1.txt")
    bays_tp2 = load_bays("baytp2.txt")
except Exception as e:
    print(f"Error loading bay files: {e}")
    raise

# Load shelves.txt
def load_shelves(filename):
    shelves = []
    try:
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                values = line.split()
                if len(values) != 7:
                    raise ValueError(f"Invalid line in {filename}: '{line}' (expected 7 values)")
                num, thickness, position, top_gap, left_gap, inter_gap, right_gap = map(int, values)
                shelves.append({
                    "number": num, "thickness": thickness, "position": position,
                    "top_gap": top_gap, "left_gap": left_gap, "inter_gap": inter_gap, "right_gap": right_gap
                })
        if not shelves:
            raise ValueError(f"{filename} is empty")
        print(f"Loaded {len(shelves)} shelves from {filename}")
        return shelves
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find {filename} in {os.getcwd()}")

try:
    shelves = load_shelves("shelves.txt")
except Exception as e:
    print(f"Error loading shelves: {e}")
    raise

# Load products.txt
def load_products(filename):
    products = []
    product_instances = []
    try:
        with open(filename, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                values = line.split()
                if len(values) != 5:
                    raise ValueError(f"Invalid line in {filename}: '{line}' (expected 5 values)")
                family, qty, length, width, height = map(int, values)
                prod = {"family": family, "length": length, "width": width, "height": height}
                products.append(prod)
                for _ in range(qty):
                    product_instances.append(prod)
        if not products:
            raise ValueError(f"{filename} is empty")
        print(f"Loaded {len(products)} products, {len(product_instances)} instances from {filename}")
        return products, product_instances
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find {filename} in {os.getcwd()}")

try:
    products, product_instances = load_products("products.txt")
    product_instances = product_instances[:100]
    print(f"Using {len(product_instances)} product instances (subset)")
except Exception as e:
    print(f"Error loading products: {e}")
    raise

# Define objectives (minimize all), avoid redefinition
if not hasattr(creator, 'FitnessMulti'):
    creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, -1.0))
if not hasattr(creator, 'Individual'):
    creator.create("Individual", list, fitness=creator.FitnessMulti)

# Get dimensions based on orientation
def get_dimensions(prod, orientation):
    l, w, h = prod["length"], prod["width"], prod["height"]
    if orientation == 1: return [l, h, w]
    if orientation == 2: return [w, l, h]
    if orientation == 3: return [w, h, l]
    if orientation == 4: return [h, l, w]
    if orientation == 5: return [h, w, l]
    return [l, w, h]

# Bottom-left packing for a shelf
def bottom_left_pack(products_on_shelf, bay_width, bay_depth, inter_gap, bay_idx):
    placed = []
    max_width = bay_width - 10 - 10
    max_depth = bay_depth
    occupied = []
    for item in products_on_shelf:
        if len(item) != 5:
            print(f"Invalid input to bottom_left_pack: {item}")
            return None
        prod, shelf, x, y, orient = item
        dims = get_dimensions(prod, orient)
        w, d = dims[0], dims[1]
        x, y = 0, 0
        while True:
            overlap = False
            for ox1, oy1, ox2, oy2 in occupied:
                if not (x + w + inter_gap <= ox1 or x >= ox2 + inter_gap or
                        y + d + inter_gap <= oy1 or y >= oy2 + inter_gap):
                    overlap = True
                    break
            if not overlap and x + w <= max_width and y + d <= max_depth:
                placed.append([prod, bay_idx, shelf, x, y, orient])
                occupied.append([x, y, x + w, y + d])
                break
            x += 10
            if x + w > max_width:
                x = 0
                y += 10
            if y + d > max_depth:
                return None
    return placed

# Greedy baseline
def greedy_baseline(bays):
    assignments = []
    current_bay = 0
    current_shelf = 0
    shelf_products = []
    for prod in product_instances:
        placed = False
        for orient in range(6):
            dims = get_dimensions(prod, orient)
            if dims[0] <= bays[current_bay]["width"] - 20 and dims[1] <= bays[current_bay]["depth"]:
                shelf_products.append([prod, current_shelf, 0, 0, orient])
                packed = bottom_left_pack(shelf_products, bays[current_bay]["width"], bays[current_bay]["depth"], 10, current_bay)
                if packed:
                    assignments.extend(packed)
                    shelf_products = [[p[0], p[2], p[3], p[4], p[5]] for p in packed]  # Convert to 5-element for next iteration
                    placed = True
                    break
        if not placed:
            shelf_products = []
            current_shelf += 1
            if current_shelf >= len(shelves):
                current_shelf = 0
                current_bay += 1
                if current_bay >= len(bays):
                    break
            shelf_products.append([prod, current_shelf, 0, 0, 0])
            packed = bottom_left_pack(shelf_products, bays[current_bay]["width"], bays[current_bay]["depth"], 10, current_bay)
            if packed:
                assignments.extend(packed)
                shelf_products = [[p[0], p[2], p[3], p[4], p[5]] for p in packed]
    individual = creator.Individual(assignments)
    return individual

# Create individual for MOGA
def create_individual(bays):
    individual = []
    assignments = {}
    for i, prod in enumerate(product_instances):
        bay = random.randint(0, len(bays)-1)
        shelf = random.randint(0, len(shelves)-1)
        x = random.uniform(0, bays[bay]["width"] - 20)
        y = random.uniform(0, bays[bay]["depth"])
        orient = random.randint(0, 5)
        key = (bay, shelf)
        if key not in assignments:
            assignments[key] = []
        assignments[key].append([prod, shelf, x, y, orient])
    
    for bay, shelf in assignments:
        packed = bottom_left_pack(assignments[(bay, shelf)], bays[bay]["width"], bays[bay]["depth"], 10, bay)
        if packed:
            individual.extend(packed)
    
    return creator.Individual(individual)

# Evaluate objectives
def evaluate(individual, bays):
    if not individual:
        return float('inf'), float('inf'), float('inf')
    used_shelves = set((ind[1], ind[2]) for ind in individual)
    num_shelves = len(used_shelves)
    
    shelf_weights = {}
    for ind in individual:
        if len(ind) != 6:
            print(f"Invalid assignment in evaluate: {ind}")
            return float('inf'), float('inf'), float('inf')
        prod, bay, shelf, x, y, orient = ind
        dims = get_dimensions(prod, orient)
        weight = dims[0] * dims[1] * dims[2]
        shelf_key = (bay, shelf)
        shelf_weights[shelf_key] = shelf_weights.get(shelf_key, 0) + weight
    weights = list(shelf_weights.values())
    weight_imbalance = np.var(weights) if weights else 0
    
    overhang = 0
    for bay, shelf in used_shelves:
        shelf_info = shelves[shelf]
        max_height = 0
        for ind in individual:
            if ind[1] == bay and ind[2] == shelf:
                dims = get_dimensions(ind[0], ind[5])
                max_height = max(max_height, dims[2])
        overhang += shelf_info["position"] - max_height
    
    return num_shelves, weight_imbalance, overhang

# Constraint check
def feasible(individual, bays):
    if not individual:
        return False
    for ind in individual:
        if len(ind) != 6:
            print(f"Invalid assignment in feasible check: {ind}")
            return False
        prod, bay, shelf, x, y, orient = ind
        shelf_info = shelves[shelf]
        dims = get_dimensions(prod, orient)
        w, d, h = dims[0], dims[1], dims[2]
        max_width = bays[bay]["width"] - shelf_info["left_gap"] - shelf_info["right_gap"]
        if x + w > max_width or y + d > bays[bay]["depth"]:
            return False
        if shelf > 0:
            prev_shelf = shelves[shelf-1]
            max_height_prev = 0
            for ind2 in individual:
                if ind2[1] == bay and ind2[2] == shelf-1:
                    dims2 = get_dimensions(ind2[0], ind2[5])
                    max_height_prev = max(max_height_prev, dims2[2])
            if prev_shelf["position"] + max_height_prev > shelf_info["position"] - 55:
                return False
        if shelf == len(shelves)-1 and shelf_info["position"] + h > bays[bay]["avail_height"]:
            return False
    return True

# Mutation
def mutate_individual(individual, bays):
    for i in range(len(individual)):
        if random.random() < 0.05:
            bay = random.randint(0, len(bays)-1)
            individual[i][1] = bay
            individual[i][2] = random.randint(0, len(shelves)-1)
            individual[i][3] = random.uniform(0, bays[bay]["width"] - 20)
            individual[i][4] = random.uniform(0, bays[bay]["depth"])
            individual[i][5] = random.randint(0, 5)
    return individual,

# Visualize Pareto front
def plot_pareto_front(hof, title, filename):
    num_shelves = [ind.fitness.values[0] for ind in hof]
    weight_imbalance = [ind.fitness.values[1] for ind in hof]
    overhang = [ind.fitness.values[2] for ind in hof]
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(num_shelves, weight_imbalance, overhang)
    ax.set_xlabel('Number of Shelves')
    ax.set_ylabel('Weight Imbalance')
    ax.set_zlabel('Height Overhang')
    ax.set_title(title)
    plt.savefig(filename, dpi=300)
    plt.close()

# Visualize shelf layout
def plot_shelf_layout(individual, bays, filename):
    if not individual:
        print(f"Cannot plot shelf layout: Empty individual")
        return
    plt.figure(figsize=(8, 6))
    bay = individual[0][1]
    for ind in individual:
        if ind[1] == bay:
            prod, _, shelf, x, y, orient = ind
            dims = get_dimensions(prod, orient)
            w, d = dims[0], dims[1]
            plt.gca().add_patch(plt.Rectangle((x, y), w, d, fill=None, edgecolor='blue'))
    plt.xlim(0, bays[bay]["width"] - 20)
    plt.ylim(0, bays[bay]["depth"])
    plt.title(f"Shelf Layout (Bay {bay})")
    plt.xlabel("Width")
    plt.ylabel("Depth")
    plt.savefig(filename, dpi=300)
    plt.close()

# MOGA setup
def run_moga(bays, pop_size=50, ngen=100):
    toolbox = base.Toolbox()
    toolbox.register("individual", create_individual, bays)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate, bays=bays)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", mutate_individual, bays=bays)
    toolbox.register("select", tools.selNSGA2)

    pop = toolbox.population(n=pop_size)
    hof = tools.ParetoFront()
    
    pop = [ind for ind in pop if feasible(ind, bays)]
    if not pop:
        print("No feasible individuals in initial population, regenerating...")
        pop = toolbox.population(n=pop_size)
    
    algorithms.eaMuPlusLambda(pop, toolbox, mu=pop_size, lambda_=pop_size, cxpb=0.8, mutpb=0.2, ngen=ngen, halloffame=hof, verbose=True)
    
    return hof

# Run experiments
def run_experiments():
    try:
        print("Running MOGA for baytp1...")
        hof_tp1 = run_moga(bays_tp1)
        if hof_tp1:
            plot_pareto_front(hof_tp1, "Pareto Front for baytp1", "pareto_tp1.png")
            plot_shelf_layout(hof_tp1[0], bays_tp1, "shelf_layout_tp1.png")
        else:
            print("No solutions found for baytp1 MOGA")
        
        print("Running greedy baseline for baytp1...")
        greedy_ind_tp1 = greedy_baseline(bays_tp1)
        greedy_fitness_tp1 = evaluate(greedy_ind_tp1, bays_tp1)
        
        print("Running MOGA for baytp2...")
        hof_tp2 = run_moga(bays_tp2)
        if hof_tp2:
            plot_pareto_front(hof_tp2, "Pareto Front for baytp2", "pareto_tp2.png")
            plot_shelf_layout(hof_tp2[0], bays_tp2, "shelf_layout_tp2.png")
        else:
            print("No solutions found for baytp2 MOGA")
        
        print("Running greedy baseline for baytp2...")
        greedy_ind_tp2 = greedy_baseline(bays_tp2)
        greedy_fitness_tp2 = evaluate(greedy_ind_tp2, bays_tp2)
        
        return hof_tp1, greedy_fitness_tp1, hof_tp2, greedy_fitness_tp2
    except Exception as e:
        print(f"Error during experiments: {e}")
        raise

# Execute and print results
try:
    hof_tp1, greedy_fitness_tp1, hof_tp2, greedy_fitness_tp2 = run_experiments()
    print("\nbaytp1 MOGA Pareto Front:")
    for ind in hof_tp1:
        print(f"Solution: {ind.fitness.values}")
    print(f"baytp1 Greedy Baseline: {greedy_fitness_tp1}")
    print("\nbaytp2 MOGA Pareto Front:")
    for ind in hof_tp2:
        print(f"Solution: {ind.fitness.values}")
    print(f"baytp2 Greedy Baseline: {greedy_fitness_tp2}")
except Exception as e:
    print(f"Execution failed: {e}")

Loaded 350 bays from baytp1.txt
Loaded 350 bays from baytp2.txt
Loaded 49 shelves from shelves.txt
Loaded 6000 products, 17793 instances from products.txt
Using 100 product instances (subset)
Running MOGA for baytp1...
No feasible individuals in initial population, regenerating...
gen	nevals
0  	50    
1  	50    
2  	50    
3  	50    
4  	50    
5  	50    
6  	50    
7  	50    
8  	50    
9  	50    
10 	50    
11 	50    
12 	50    
13 	50    
14 	50    
15 	50    
16 	50    
17 	50    
18 	50    
19 	50    
20 	50    
21 	50    
22 	50    
23 	50    
24 	50    
25 	50    
26 	50    
27 	50    
28 	50    
29 	50    
30 	50    
31 	50    
32 	50    
33 	50    
34 	50    
35 	50    
36 	50    
37 	50    
38 	50    
39 	50    
40 	50    
41 	50    
42 	50    
43 	50    
44 	50    
45 	50    
46 	50    
47 	50    
48 	50    
49 	50    
50 	50    
51 	50    
52 	50    
53 	50    
54 	50    
55 	50    
56 	50    
57 	50    
58 	50    
59 	50    
60 	50    
61 	50    
62 	50    
63 	50    
64 